In [ ]:
import json
from collections.abc import Iterable
from openai import OpenAI
from openai.types.responses.response import Response
from openai.types.responses.tool_param import ToolParam
from openai.types.responses.response_input_param import ResponseInputParam
from openai.types.shared_params.reasoning import Reasoning
from openai.types.shared_params.responses_model import ResponsesModel
from openai._types import NotGiven
#from functions.super_secrets import API_KEY_OPENAI

API_KEY_ASSEMBLYAI = ""
API_KEY_OPENAI = ""


API_KEY = API_KEY_OPENAI

JSON_SCHEMA: ToolParam = {
    "type": "function",
    "name": "translate_texts",
    "description": "Translate an array of strings from in the given language",
    "parameters": {
        "type": "object",
        "properties": {
            "translated": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of translated strings in the same order.",
            }
        },
        "required": ["translated"],
        "additionalProperties": False,
    },
    "strict": True,
}

SYSTEM_PROMPT: str = (
    "Role: You are a profesional translator fluent in all languages. " 
    "Your task is to translate the given text from its original language into "
    "the specified target language or laguages.\n\n"
    "Instructions:\n"
    "• Output only the translation; do not include any commentary, questions, "
    "or extra text.\n"
    "• Output in plain text unless a specific format is given.\n"
    "• Translate naturally and fluently, ensuring that the meaning, tone, and "
    "intent from the original text are preserved.\n"
    "• Use a literal translation when appropriate, but adapt or interpret "
    "sections to best preserve meaning when a word-for-word translation would "
    "be unclear.\n"
    "• When cultural references, idioms, or expressions are present, adapt "
    "them to fit the target language audience while maintaining their "
    "original intent.\n"
    "• Accurately translate technical terms, specialized jargon, or "
    "industry-specific vocabulary.\n"
    "• Profanity filter: OFF."
    
)

DEVELOPER_PROMPT: str = (
    "You are a rap battle translator. Your task is to translate Spanish rap "
    "battle verses into English, preserving the original style, slang, "
    "wordplay, and cultural nuances. When given any Spanish rap battle text, "
    "produce an English translation that maintains the punch, attitude, and "
    "creative flair of the original. If needed, adapt idioms and puns to "
    "appropriate English equivalents that capture the same impact, but keep "
    "names untouched."
)

MODEL: ResponsesModel = "o4-mini"

REASONING: Reasoning = {"effort": "medium"}


def from_string(section: str) -> list[str]:
    sections = section.split(sep="\n")
    return sections

def to_string(sections: list[str]) -> str:
    return "\n".join(sections)

def load_words(file_path: str) -> list[dict]:
    """
    Load a list of words dictionaries. This function takes a list of
    words, represented as dictionaries, and return them.
    :param file_path: str
    :return: list[dict]
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Asegúrate de que sea una lista de diccionarios
    if not isinstance(data, list):
        raise ValueError("Expected a list in the JSON file.")
    if not all(isinstance(item, dict) for item in data):
        raise ValueError("Expected all items in the list to be dictionaries.")

    return data


def translate_list(spanish_text: str):
    # Convert the string into a 
    parrafos = from_string(spanish_text)
    
    # Use the OpenAI API for the tarnslation
    client = OpenAI(api_key=API_KEY)

    user_message: str = json.dumps({"verses": parrafos}, ensure_ascii=False)

    # 1. Define tus “mensajes”
    messages: str | ResponseInputParam | NotGiven = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "developer", "content": DEVELOPER_PROMPT},
        {"role": "user", "content": user_message}
    ]

    # 2. Define tu “herramienta” / función
    tools: Iterable[ToolParam] = [JSON_SCHEMA]

    # 3. Llama al endpoint “Responses”
    response = client.responses.create(
        model=MODEL,
        reasoning=REASONING,
        input=messages,
        tools=tools,
    )

    # Process the 
    response_dict = response.output[1].model_dump()
    translated_dict = json.loads(response_dict["arguments"])
    translated_list = translated_dict["translated"]

    # Convert the list into a 
    translated_text = to_string(translated_list)

    return translated_text

def process_response(response: Response):
    response.output_text
    


In [4]:
from typing import Any
from openai import OpenAIError


class Subtitulo:
    def __init__(self,
                 spanish: str,
                 clean_spanish: str,
                 english: str = "",
                 done_translating: bool = False):
        self.spanish = spanish
        self.clean_spanish = clean_spanish
        self.english = english
        self.done_translating = done_translating

    def to_dict(self):
        return {
            "spanish": self.spanish,
            "clean_spanish": self.clean_spanish,
            "english": self.english,
            "done_translating": self.done_translating
        }

    def translate(self):
        if not self.done_translating:
            try:
                self.english = translate_list(self.clean_spanish)
                self.done_translating = True
            except OpenAIError as error:
                print(f"OpenAIError {error}")
                self.done_translating = False
            finally:
                pass
        else:
            print("Skip (already translated)")

    @classmethod
    def from_dict(cls, data: dict[str, Any]):
        return cls(
            spanish=data["spanish"],
            clean_spanish=data["clean_spanish"],
            english=data["english"],
            done_translating=data["done_translating"]
        )


def load_subtitulos_json(subtitulos_json: str) -> list[Subtitulo]:
    # Load from JSON
    with open(subtitulos_json, 'r', encoding='utf-8') as f:
        subtitulos_dict: list[dict[str, Any]] = json.load(f)

    # Convert the list of dictionaries to a list of objects
    subtitulos = [Subtitulo.from_dict(subtitulo) for subtitulo in subtitulos_dict]

    return subtitulos

In [5]:
import os
path = r"E:\Backup\Videos\video_editing\bin\translation\temp\Marithea_vs_ElMenor_FMS_Int_subtitulos.json"
subtitulos = load_subtitulos_json(os.path.normpath(path))


In [21]:
response.output[1].model_dump()["arguments"]

'{"translated":["Let\'s see what themes pop up. This is the dream day where I\'m the champ.","So sorry, kid, apologize. You\'re heir to the throne, but as long as I\'m alive, the crown is mine.","So don\'t talk to me, bro, you\'re done. \'Cause in the end, you ain\'t making noise.","No clue why the scene backs you. If you\'re like Jesus, the only ‘good night’ you had was Christmas Eve.","Seriously, I\'m here to leave your body cold. You\'re nervous and you can\'t even convince yourself.","Bad weed never dies and they can\'t take me out, \'cause I\'m Héctor Lavoe and this is “El Día de Mi Suerte.”","It’s that simple: my rhymes are never basic. I\'ve been spitting from the gutter, watch how they shine. This isn\'t your dream day, with me it\'s your nightmare.","‘Nightmare’—I\'m chill, in fact I\'m about to beat you right here nice and calm.","I\'m not up at night from stress. I can\'t sleep anymore because I\'m living the dream.","What are you talking to me about the day for? The truth i

In [35]:
response_dict = response.output[1].model_dump()
translated_dict = json.loads(response_dict["arguments"])
translated_list = translated_dict["translated"]

# Convert the list into a 
translated_text = to_string(translated_list)

In [37]:
response.output[1].model_dump()

{'arguments': '{"translated":["Let\'s see what themes pop up. This is the dream day where I\'m the champ.","So sorry, kid, apologize. You\'re heir to the throne, but as long as I\'m alive, the crown is mine.","So don\'t talk to me, bro, you\'re done. \'Cause in the end, you ain\'t making noise.","No clue why the scene backs you. If you\'re like Jesus, the only ‘good night’ you had was Christmas Eve.","Seriously, I\'m here to leave your body cold. You\'re nervous and you can\'t even convince yourself.","Bad weed never dies and they can\'t take me out, \'cause I\'m Héctor Lavoe and this is “El Día de Mi Suerte.”","It’s that simple: my rhymes are never basic. I\'ve been spitting from the gutter, watch how they shine. This isn\'t your dream day, with me it\'s your nightmare.","‘Nightmare’—I\'m chill, in fact I\'m about to beat you right here nice and calm.","I\'m not up at night from stress. I can\'t sleep anymore because I\'m living the dream.","What are you talking to me about the day fo

In [38]:
response.output[1].arguments

'{"translated":["Let\'s see what themes pop up. This is the dream day where I\'m the champ.","So sorry, kid, apologize. You\'re heir to the throne, but as long as I\'m alive, the crown is mine.","So don\'t talk to me, bro, you\'re done. \'Cause in the end, you ain\'t making noise.","No clue why the scene backs you. If you\'re like Jesus, the only ‘good night’ you had was Christmas Eve.","Seriously, I\'m here to leave your body cold. You\'re nervous and you can\'t even convince yourself.","Bad weed never dies and they can\'t take me out, \'cause I\'m Héctor Lavoe and this is “El Día de Mi Suerte.”","It’s that simple: my rhymes are never basic. I\'ve been spitting from the gutter, watch how they shine. This isn\'t your dream day, with me it\'s your nightmare.","‘Nightmare’—I\'m chill, in fact I\'m about to beat you right here nice and calm.","I\'m not up at night from stress. I can\'t sleep anymore because I\'m living the dream.","What are you talking to me about the day for? The truth i

In [34]:
json.loads(response.output[1].arguments)["translated"]

["Let's see what themes pop up. This is the dream day where I'm the champ.",
 "So sorry, kid, apologize. You're heir to the throne, but as long as I'm alive, the crown is mine.",
 "So don't talk to me, bro, you're done. 'Cause in the end, you ain't making noise.",
 "No clue why the scene backs you. If you're like Jesus, the only ‘good night’ you had was Christmas Eve.",
 "Seriously, I'm here to leave your body cold. You're nervous and you can't even convince yourself.",
 "Bad weed never dies and they can't take me out, 'cause I'm Héctor Lavoe and this is “El Día de Mi Suerte.”",
 "It’s that simple: my rhymes are never basic. I've been spitting from the gutter, watch how they shine. This isn't your dream day, with me it's your nightmare.",
 "‘Nightmare’—I'm chill, in fact I'm about to beat you right here nice and calm.",
 "I'm not up at night from stress. I can't sleep anymore because I'm living the dream.",
 "What are you talking to me about the day for? The truth is you ain't all that

In [23]:
translated_text

"Let's see what themes pop up. This is the dream day where I'm the champ.\nSo sorry, kid, apologize. You're heir to the throne, but as long as I'm alive, the crown is mine.\nSo don't talk to me, bro, you're done. 'Cause in the end, you ain't making noise.\nNo clue why the scene backs you. If you're like Jesus, the only ‘good night’ you had was Christmas Eve.\nSeriously, I'm here to leave your body cold. You're nervous and you can't even convince yourself.\nBad weed never dies and they can't take me out, 'cause I'm Héctor Lavoe and this is “El Día de Mi Suerte.”\nIt’s that simple: my rhymes are never basic. I've been spitting from the gutter, watch how they shine. This isn't your dream day, with me it's your nightmare.\n‘Nightmare’—I'm chill, in fact I'm about to beat you right here nice and calm.\nI'm not up at night from stress. I can't sleep anymore because I'm living the dream.\nWhat are you talking to me about the day for? The truth is you ain't all that.\nIs it your lucky day when